# Google Earth Engine imagery download tester

Sometimes during development you can make a change that suddenly breaks all of your imagery-downloading ability. You may not know whether it is your code or the Google servers that are to blame. The following code block downloads and plots a bit of imagery to ensure that the Google servers are, indeed, working as they should.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pprint
pp = pprint.PrettyPrinter(indent=4)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ee
import random, os, zipfile, requests, StringIO



def unzipURL(img_url, tmp_directory = None):
    """
    Downloads and unzips a file in a tmp directory.
    Hits the server once.

    img_url: A web address to a zip file.
    tmp_directory: Optional path to unzip the contents.

    Returns: Path to the directory containing the contents of the zip file.
    """

    #
    # Set up temp directory
    #
    if tmp_directory == None:
        tmp_directory="./tmp/"+str(random.randrange(99999999))
    zip_filename = os.path.join(tmp_directory, 'zipped_tile.zip')
    # make tmp directory
    if not os.path.exists(tmp_directory):
        os.makedirs(tmp_directory)
    # and delete everything in it:
    filelist = [ f for f in os.listdir(tmp_directory)]
    for f in filelist:
        os.remove(os.path.join(tmp_directory, f))

    #
    # Download file
    #
    try:
        r = requests.get(img_url)
        sc = r.status_code
    except requests.ConnectionError as e:
        raise e

    if r.status_code == 200:
        z = zipfile.ZipFile(StringIO.StringIO(r.content))
        z.extractall(tmp_directory)
        z.close()
        return tmp_directory
    elif r.status_code == 400:
        print "Uh-oh. Status code 400 (Bad Request). Possible problems:"
        print "- You requested a band that wasn't available in every image."
        print "- Some other unknown issue."
        raise ServerError("Status code 400")
    elif r.status_code == 429:
        raise ServerError("Status code 429")
    else:
        raise ServerError("Uh-oh. Status code " + str(r.status_code))

    # Fall through on handled error/exception
    raise Exception("Unknown error in unzipURL().")



ee.Initialize()

maps_collection = ee.ImageCollection('USDA/NAIP/DOQQ')

# create the Google Earth Engine Image object
gee_img = ee.Image(maps_collection.first())

# get the download link from google
path=gee_img.getDownloadUrl({
        'name': 'map_section',  # name the file (otherwise it will be a uninterpretable hash)
        'scale': 100,                              # resolution in meters
        'crs': 'EPSG:4326', #4326                         #  projection
        'bands': [{'id': 'R'}]
        });

# convert to tiff
tif_location = unzipURL(path)

# get handles to the tiff files

tif_path =os.path.join(tif_location, 'map_section.R.tif')

img=mpimg.imread(tif_path)
plt.imshow(img)
plt.show()





# Rate Limit Tester

Google Earth Engine limits api calls to 3 per second. This next bit of code tests that and makes sure we are getting the level of service we expect.

In [ ]:
import ee, time, requests, random
from threading import Thread
from tqdm import trange
ee.Initialize()

request_status = []
threads = []

def make_call(i):
    """ Downloads a random 400x400 meter patch of Brooklyn.
    Hits the server 3 times. """
    
    # get random 400x400 meter box
    bounds = ee.Geometry.Point([random.uniform(-73.965471, -73.920207), random.uniform(40.614974, 40.693991)]).buffer(200).bounds()
    gee_img = ee.ImageCollection('USDA/NAIP/DOQQ').filterBounds(bounds).select(['R', 'G', 'B']).mosaic()
    
    url=gee_img.getDownloadUrl({ # SERVER HIT 2
        'scale': 100,           
        'crs': 'EPSG:4326', 
        'bands': [{'id': 'R'}, {'id': 'G'}, {'id': 'B'}],
        'region': bounds.getInfo()['coordinates'] # SERVER HIT 1
        });
    
    r = requests.get(url) # SERVER HIT 3
    request_status.append(r.status_code)

# spawn download threads
for i in trange(100):
    t = Thread(target=make_call, args=(i,))
    t.start()
    threads.append(t)
    time.sleep(2)

# wait for threads to finish
for t in threads:
    t.join()
    

print request_status
print 100 * request_status.count(200) / float(len(request_status)), "% success."

